# Testing for the models module

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import numpy as np
from fastcore.test import *
from circadian.models import *
from scipy.signal import find_peaks
from circadian.lights import LightSchedule

# DinamicalTrajectory

In [ ]:
# test DynamicalTrajectory's constructor
total_timepoints = 1000
variables = 2
time = np.linspace(0, 10, total_timepoints)
states = np.zeros((total_timepoints, variables))
states[:, 0] = np.sin(time)
states[:, 1] = np.cos(time)
traj = DynamicalTrajectory(time, states)
# test attributes
test_eq(traj.num_states, variables)
test_eq(traj.batch_size, 1)
test_eq(traj.time, time)
test_eq(traj.states, states)
# test batch handling
batches = 5
batch_states = np.zeros((total_timepoints, variables, batches))
batch_states[:, 0, :] = np.sin(time)[:, None]
batch_states[:, 1, :] = np.cos(time)[:, None]
batch_traj = DynamicalTrajectory(time, batch_states)
test_eq(batch_traj.num_states, variables)
test_eq(batch_traj.batch_size, batches)
test_eq(batch_traj.time, time)
test_eq(batch_traj.states, batch_states)
# test error handling
test_fail(lambda: DynamicalTrajectory(1, np.array([1, 2])), contains="time must be a numpy array")
test_fail(lambda: DynamicalTrajectory(np.array([[1, 2], [1, 2]]), np.array([1, 2]) ), contains="time must be a 1D array")
test_fail(lambda: DynamicalTrajectory(np.array(["1", "2"]), np.array([1, 2])), contains="time must be numeric")
test_fail(lambda: DynamicalTrajectory(np.array([2, 1]), np.array([1, 2])), contains="time must be monotonically increasing")
test_fail(lambda: DynamicalTrajectory(np.array([1, 2]), 1), contains="states must be a numpy array")
test_fail(lambda: DynamicalTrajectory(np.array([1, 2]), np.array(1)), contains="states must have at least 1 dimension")
test_fail(lambda: DynamicalTrajectory(np.array([1, 2]), np.zeros((1,1,1,1))), contains="states must have at most 3 dimensions")
test_fail(lambda: DynamicalTrajectory(np.array([1, 2, 3]), np.array([1, 2])), contains="states must have the same length as time")
test_fail(lambda: DynamicalTrajectory(np.array([1, 2]), np.array([1, 2, 3])), contains="states must have the same length as time")
test_fail(lambda: DynamicalTrajectory(np.array([1, 2]), np.array(["1", "2"])), contains="states must be numeric")

In [ ]:
# test DynamicalTrajectory's __call__
total_timepoints = 1000
variables = 2
time = np.linspace(0, 10, total_timepoints)
states = np.zeros((total_timepoints, variables))
states[:, 0] = np.sin(time)
states[:, 1] = np.cos(time)
traj = DynamicalTrajectory(time, states)
interpolation_error = np.abs(np.sum(traj(5.0) - (np.sin(5.0), np.cos(5.0))))
test_eq(interpolation_error < 1e-4, True)
# handle batch
batches = 5
batch_states = np.zeros((total_timepoints, variables, batches))
batch_states[:, 0, :] = np.sin(time)[:, None]
batch_states[:, 1, :] = np.cos(time)[:, None]
batch_traj = DynamicalTrajectory(time, batch_states)
interpolation_error = np.abs(np.sum((batch_traj(0.5)[0] - np.sin(0.5)) + (batch_traj(0.5)[1] - np.cos(0.5))))
test_eq(interpolation_error < 1e-4, True)
# test error handling
test_fail(lambda: traj("1"), contains="timepoint must be int or float")
test_fail(lambda: traj(11), contains="timepoint must be within the time range")

In [ ]:
# test DynamicalTrajectory's __getitem__
total_timepoints = 1000
variables = 2
time = np.linspace(0, np.pi, total_timepoints)
states = np.zeros((total_timepoints, variables))
states[:, 0] = np.sin(time)
states[:, 1] = np.cos(time)
traj = DynamicalTrajectory(time, states)
test_eq(traj[0], (0.0, np.array([0.0, 1.0])))
states = traj[-1][1]
difference = np.abs(np.sum(states - (np.sin(np.pi), np.cos(np.pi))))
test_eq(difference < 1e-4, True)
# handle batch
batches = 5
batch_states = np.zeros((total_timepoints, variables, batches))
batch_states[:, 0, :] = np.sin(time)[:, None]
batch_states[:, 1, :] = np.cos(time)[:, None]
batch_traj = DynamicalTrajectory(time, batch_states)
test_eq(batch_traj[0][0], 0.0)
test_eq(np.all(batch_traj[0][1][0]==np.zeros(batches)), True)
test_eq(np.all(batch_traj[0][1][1]==np.ones(batches)), True)
# test error handling
test_fail(lambda: traj["1"], contains="idx must be int")
test_fail(lambda: traj[-2], contains="idx must be within 0 and")
test_fail(lambda: traj[1000], contains="idx must be within 0 and")

In [ ]:
# test DynamicalTrajectory's __len__
total_timepoints = 1000
variables = 2
time = np.linspace(0, np.pi, total_timepoints)
states = np.zeros((total_timepoints, variables))
states[:, 0] = np.sin(time)
states[:, 1] = np.cos(time)
traj = DynamicalTrajectory(time, states)
test_eq(len(traj), total_timepoints)
# handle batch
batches = 5
batch_states = np.zeros((total_timepoints, variables, batches))
batch_states[:, 0, :] = np.sin(time)[:, None]
batch_states[:, 1, :] = np.cos(time)[:, None]
batch_traj = DynamicalTrajectory(time, batch_states)
test_eq(len(batch_traj), total_timepoints)

In [ ]:
# test DynamicalTrajectory's get_batch
total_timepoints = 1000
variables = 2
time = np.linspace(0, np.pi, total_timepoints)
states = np.zeros((total_timepoints, variables))
states[:, 0] = np.sin(time)
states[:, 1] = np.cos(time)
traj = DynamicalTrajectory(time, states)
test_eq(traj.get_batch(0).batch_size, 1)
test_eq(traj.get_batch(0).num_states, variables)
test_eq(traj.get_batch(0).time, time)
test_eq(traj.get_batch(0).states, states)
test_eq(traj.get_batch(0).states.ndim, 2)
test_eq(traj.get_batch(0).states.shape, (total_timepoints, variables))
# handle batch
batches = 5
batch_states = np.zeros((total_timepoints, variables, batches))
batch_states[:, 0, :] = np.sin(time)[:, None]
batch_states[:, 1, :] = np.cos(time)[:, None]
batch_traj = DynamicalTrajectory(time, batch_states)
test_eq(batch_traj.get_batch(0).batch_size, 1)
test_eq(batch_traj.get_batch(0).num_states, variables)
test_eq(batch_traj.get_batch(0).time, time)
test_eq(batch_traj.get_batch(0).states, batch_states[:, :, 0])
test_eq(batch_traj.get_batch(0).states.ndim, 2)
test_eq(batch_traj.get_batch(0).states.shape, (total_timepoints, variables))
test_eq(batch_traj.get_batch(4).batch_size, 1)
test_eq(batch_traj.get_batch(4).num_states, variables)
test_eq(batch_traj.get_batch(4).time, time)
test_eq(batch_traj.get_batch(4).states, batch_states[:, :, 0])
test_eq(batch_traj.get_batch(4).states.ndim, 2)
test_eq(batch_traj.get_batch(4).states.shape, (total_timepoints, variables))
test_eq(batch_traj.get_batch(-1).states, batch_traj.get_batch(4).states)
# test error handling
test_fail(lambda: traj.get_batch("1"), contains="batch_idx must be int")
test_fail(lambda: traj.get_batch(5), contains="batch_idx must be within -1 and")
test_fail(lambda: traj.get_batch(-2), contains="batch_idx must be within -1 and")

# CircadianModel

In [ ]:
from circadian.models import _initial_condition_input_checking, _wake_input_checking
from circadian.models import _model_input_checking, _light_input_checking

In [ ]:
# test input checking with NaNs
initial_condition_nan = np.array([np.nan, 0.0, 0.0])
test_fail(lambda: _initial_condition_input_checking(
    initial_condition_nan,
    3
), contains="initial_condition must not contain NaNs")
input_nan = np.array([[np.nan, 0.0, 0.0]])
test_fail(lambda: _model_input_checking(
    input_nan,
    3,
    [1]
))
light_nan = np.array([np.nan, 0.0, 0.0])
test_fail(lambda: _light_input_checking(light_nan), contains="light must not contain NaNs")
wake_nan = np.array([np.nan, 0.0, 0.0])
test_fail(lambda: _wake_input_checking(wake_nan), contains="wake must not contain NaNs")

In [ ]:
# test CircadianModel's constructor
default_params = {"a": 1, "b": 2}
num_states = 3
num_inputs = 2
default_initial_condition = np.array([1, 2, 3])
model = CircadianModel(default_params, num_states, num_inputs, default_initial_condition)
# test attributes
test_eq(model._default_params, default_params)
test_eq(model._num_states, num_states)
test_eq(model._num_inputs, num_inputs)
test_eq(model._default_initial_condition, default_initial_condition)
test_eq(model.parameters, default_params)
test_eq(model.initial_condition, default_initial_condition)
test_eq(model.a, 1)
test_eq(model.b, 2)
test_eq(model.trajectory, None)
# test error handling
test_fail(lambda: CircadianModel(1, num_states, num_inputs, default_initial_condition), contains="parameters must be a dictionary")
test_fail(lambda: CircadianModel({}, num_states, num_inputs, default_initial_condition), contains="parameters must not be empty")
test_fail(lambda: CircadianModel({1: 1, 2:2}, num_states, num_inputs, default_initial_condition), contains="keys of parameters must be strings")
test_fail(lambda: CircadianModel({"a": "1", "b":2}, num_states, num_inputs, default_initial_condition), contains="values of parameters must be numeric")
test_fail(lambda: CircadianModel(default_params, "1", num_inputs, default_initial_condition), contains="num_states must be an integer")
test_fail(lambda: CircadianModel(default_params, -1, num_inputs, default_initial_condition), contains="num_states must be positive")
test_fail(lambda: CircadianModel(default_params, num_states, "1", default_initial_condition), contains="num_inputs must be an integer")
test_fail(lambda: CircadianModel(default_params, num_states, -1, default_initial_condition), contains="num_inputs must be positive")
test_fail(lambda: CircadianModel(default_params, num_states, num_inputs, "1"), contains="initial_condition must be a numpy array")
test_fail(lambda: CircadianModel(default_params, num_states, num_inputs, np.array(["1", "2", "3"])), contains="initial_condition must be numeric")
test_fail(lambda: CircadianModel(default_params, num_states, num_inputs, np.array([1, 2, 3, 4])), contains="initial_condition must have length")

In [ ]:
# test CircadianModel's step_rk4
default_params = {"a": 1, "b": 2}
num_states = 3
num_inputs = 2
default_initial_condition = np.array([1, 2, 3])
model = CircadianModel(default_params, num_states, num_inputs, default_initial_condition)
model.derv = lambda t, state, input: np.ones_like(state)
t = 0.0
light_input = 1.0
dt = 0.1
test_eq(np.all(model.step_rk4(t, default_initial_condition, light_input, dt) == np.array([1.1, 2.1, 3.1])), True)

In [ ]:
# test integrate
default_params = {"a": 1, "b": 2}
num_states = 3
num_inputs = 2
default_initial_condition = np.array([1, 2, 3])
model = CircadianModel(default_params, num_states, num_inputs, default_initial_condition)
time = np.linspace(0, 10, 1000)
light_input = np.ones((len(time), num_inputs))
model.derv = lambda t, state, input: np.ones_like(state)
model.integrate(time, default_initial_condition, light_input)
test_eq(model.trajectory.num_states, num_states)
test_eq(model.trajectory.batch_size, 1)
test_eq(model.trajectory.time, time)
state_0_error = np.abs(np.sum(model.trajectory.states[:, 0] - time - 1))
state_1_error = np.abs(np.sum(model.trajectory.states[:, 1] - time - 2))
state_2_error = np.abs(np.sum(model.trajectory.states[:, 2] - time - 3))
test_eq(state_0_error < 1e-10, True)
test_eq(state_1_error < 1e-10, True)
test_eq(state_2_error < 1e-10, True)
# handle batches
batches = 5
batch_initial_conditions = np.zeros((num_states, batches))
batch_initial_conditions[:, 1] = 1
batch_initial_conditions[:, 2] = 2
batch_initial_conditions[:, 3] = 3
batch_initial_conditions[:, 4] = 4
model.integrate(time, batch_initial_conditions, light_input)
for batch in range(batches):
    batch_trajectory = model.trajectory.get_batch(batch)
    test_eq(batch_trajectory.num_states, num_states)
    test_eq(batch_trajectory.time, time)
    state_0_error = np.abs(np.sum(batch_trajectory.states[:, 0] - time - batch))
    state_1_error = np.abs(np.sum(batch_trajectory.states[:, 1] - time - batch))
    state_2_error = np.abs(np.sum(batch_trajectory.states[:, 2] - time - batch))
    test_eq(state_0_error < 1e-10, True)
    test_eq(state_1_error < 1e-10, True)
    test_eq(state_2_error < 1e-10, True)
# test error handling
test_fail(lambda: model.integrate(1, default_initial_condition, light_input), contains="time must be a numpy array")
test_fail(lambda: model.integrate(np.array([[1, 2], [1, 2]]), default_initial_condition, light_input), contains="time must be a 1D array")
test_fail(lambda: model.integrate(np.array(["1", "2"]), default_initial_condition, light_input), contains="time must be numeric")
test_fail(lambda: model.integrate(np.array([2, 1]), default_initial_condition, light_input), contains="time must be monotonically increasing")
test_fail(lambda: model.integrate(time, 1, light_input), contains="initial_condition must be a numpy array")
test_fail(lambda: model.integrate(time, np.array(["1", "2", "3"]), light_input), contains="initial_condition must be numeric")
test_fail(lambda: model.integrate(time, np.array([1, 2]), light_input), contains="initial_condition must have length")
test_fail(lambda: model.integrate(time, np.array([1, 2]), light_input[:-3]), contains="input's first dimension must have length")
test_fail(lambda: model.integrate(time, default_initial_condition, 1), contains="input must be a numpy array")
test_fail(lambda: model.integrate(time, default_initial_condition, np.array(["1", "2"])), contains="input must be numeric")
test_fail(lambda: model.integrate(time, default_initial_condition), contains="a model input must be provided via the input argument")

In [ ]:
# test CircadianModel's __call__
default_params = {"a": 1, "b": 2}
num_states = 3
num_inputs = 2
default_initial_condition = np.array([1, 2, 3])
model = CircadianModel(default_params, num_states, num_inputs, default_initial_condition)
time = np.linspace(0, 10, 1000)
light_input = np.ones((len(time), num_inputs))
model.derv = lambda t, state, input: np.ones_like(state)
model(time, default_initial_condition, light_input)
test_eq(model.trajectory.num_states, num_states)
test_eq(model.trajectory.batch_size, 1)
test_eq(model.trajectory.time, time)
state_0_error = np.abs(np.sum(model.trajectory.states[:, 0] - time - 1))
state_1_error = np.abs(np.sum(model.trajectory.states[:, 1] - time - 2))
state_2_error = np.abs(np.sum(model.trajectory.states[:, 2] - time - 3))
test_eq(state_0_error < 1e-10, True)
test_eq(state_1_error < 1e-10, True)
test_eq(state_2_error < 1e-10, True)
# handle batches
batches = 5
batch_initial_conditions = np.zeros((num_states, batches))
batch_initial_conditions[:, 1] = 1
batch_initial_conditions[:, 2] = 2
batch_initial_conditions[:, 3] = 3
batch_initial_conditions[:, 4] = 4
model(time, batch_initial_conditions, light_input)
for batch in range(batches):
    batch_trajectory = model.trajectory.get_batch(batch)
    test_eq(batch_trajectory.num_states, num_states)
    test_eq(batch_trajectory.time, time)
    state_0_error = np.abs(np.sum(batch_trajectory.states[:, 0] - time - batch))
    state_1_error = np.abs(np.sum(batch_trajectory.states[:, 1] - time - batch))
    state_2_error = np.abs(np.sum(batch_trajectory.states[:, 2] - time - batch))
    test_eq(state_0_error < 1e-10, True)
    test_eq(state_1_error < 1e-10, True)
    test_eq(state_2_error < 1e-10, True)
# test error handling
test_fail(lambda: model(1, default_initial_condition, light_input), contains="time must be a numpy array")
test_fail(lambda: model(np.array([[1, 2], [1, 2]]), default_initial_condition, light_input), contains="time must be a 1D array")
test_fail(lambda: model(np.array(["1", "2"]), default_initial_condition, light_input), contains="time must be numeric")
test_fail(lambda: model(np.array([2, 1]), default_initial_condition, light_input), contains="time must be monotonically increasing")
test_fail(lambda: model(time, 1, light_input), contains="initial_condition must be a numpy array")
test_fail(lambda: model(time, np.array(["1", "2", "3"]), light_input), contains="initial_condition must be numeric")
test_fail(lambda: model(time, np.array([1, 2]), light_input), contains="initial_condition must have length")
test_fail(lambda: model(time, default_initial_condition, 1), contains="input must be a numpy array")
test_fail(lambda: model(time, default_initial_condition, light_input[:-3]), contains="input's first dimension must have length")
test_fail(lambda: model(time, default_initial_condition, np.array(["1", "2"])), contains="input must be numeric")
test_fail(lambda: model(time, default_initial_condition), contains="a model input must be provided via the input argument")

In [ ]:
# test CircadianModel's get_parameters_array
default_params = {"a": 1, "b": 2}
num_states = 3
num_inputs = 1
default_initial_conditions = np.array([1, 2, 3])
model = CircadianModel(default_params, num_states, num_inputs, default_initial_conditions)
test_eq(model.get_parameters_array(), np.array([1, 2]))

In [ ]:
# test CircadianModel's equilibrate
default_params = {"a": 1, "b": 2}
num_states = 3
num_inputs = 2
default_initial_condition = np.array([1, 2, 3])
model = CircadianModel(default_params, num_states, num_inputs, default_initial_condition)
model.derv = lambda t, state, input: np.ones_like(state)
model.dlmos = lambda: np.array([1, 2, 3])
time = np.linspace(0, 10, 1000)
light_input = np.ones((len(time), num_inputs))
model(time, default_initial_condition, light_input)
loops = 2
new_initial_condition = model.equilibrate(time, light_input, loops)
test_eq(new_initial_condition.shape, default_initial_condition.shape)
ground_truth = default_initial_condition + loops * time[-1]
test_eq(np.all(np.isclose(ground_truth, new_initial_condition)), True)
# test error handling
test_fail(lambda: model.equilibrate(1, light_input, loops), contains="time must be a numpy array")
test_fail(lambda: model.equilibrate(np.array([[1, 2], [1, 2]]), light_input, loops), contains="time must be a 1D array")
test_fail(lambda: model.equilibrate(np.array(["1", "2"]), light_input, loops), contains="time must be numeric")
test_fail(lambda: model.equilibrate(np.array([2, 1]), light_input, loops), contains="time must be monotonically increasing")
test_fail(lambda: model.equilibrate(time, 1, loops), contains="input must be a numpy array")
test_fail(lambda: model.equilibrate(time, np.array([[1, 2], [1, 2]]), loops), contains="input's first dimension must have length")
test_fail(lambda: model.equilibrate(time, np.array(["1", "2"]), loops), contains="input must be numeric")
test_fail(lambda: model.equilibrate(time, light_input, "1"), contains="num_loops must be an integer")
test_fail(lambda: model.equilibrate(time, light_input, -1), contains="num_loops must be positive")
# test equilibration warning
model.dlmos = lambda: np.array(np.random.rand(3))
test_warns(lambda: model.equilibrate(time, light_input, loops))

# Forger99

In [ ]:
# test Forger99's constructor
model = Forger99()
# test attributes
true_default_params = {
    'taux': 24.2, 'mu': 0.23, 'G': 33.75,
    'alpha_0': 0.05, 'beta': 0.0075, 'p': 0.50,
    'I0': 9500.0, 'k': 0.55, 'cbt_to_dlmo': 7.0}
test_eq(model._default_params, true_default_params)
true_initial_condition = np.array([-0.0843259, -1.09607546, 0.45584306])
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, true_default_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 1)
for key, value in true_default_params.items():
    test_eq(getattr(model, key), value)
# test initialization with custom parameters
custom_params = {
    'taux': 1.0, 'mu': 2.0, 'G': 3.0,
    'alpha_0': 4.0, 'beta': 5.0, 'p': 6.0,
    'I0': 7.0, 'k': 8.0, 'cbt_to_dlmo': 9.0}
model = Forger99(custom_params)
# test attributes
test_eq(model._default_params, true_default_params)
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, custom_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 1)
for key, value in custom_params.items():
    test_eq(getattr(model, key), value)

In [ ]:
# test Forger99's integrate input handling
model = Forger99()
time = np.array([0, 1, 2])
light_input = np.ones(len(time))
test_fail(lambda: model.integrate(time, input="1"), contains="light must be a numpy array")
test_fail(lambda: model.integrate(time, input=np.ones((1,1))), contains="light must be a 1D")
test_fail(lambda: model.integrate(time, input=np.array(["1", "2"])), contains="light must be numeric")
test_fail(lambda: model.integrate(time, input=-light_input), contains="light intensity must be nonnegative")

In [ ]:
# test Forger99's derv
model = Forger99()
# single batch
t = 0.0
state = np.array([-0.3, -1.13, 0.0])
light = 1.0
derv_error = np.abs(np.sum(model.derv(t, state, light) - np.array([-0.28846216, 0.1267787, 0.03077935])))
test_eq(derv_error < 1e-8, True)
# multiple batches
batches = 5
batch_states = np.zeros((3, batches))
for batch in range(batches):
    batch_states[:, batch] = state
batch_derv = model.derv(t, batch_states, light)
for batch in range(batches):
    derv_error = np.abs(np.sum(batch_derv[:, batch] - np.array([-0.28846216, 0.1267787, 0.03077935])))
    test_eq(derv_error < 1e-8, True)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [ ]:
# test Forger99's phase
model = Forger99()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(time=t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(trajectory, t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test calculation for all trajectories
phase = model.phase()
test_eq(np.all(np.isclose(phase, true_phase)), True)
# test error handling
test_fail(lambda: model.phase(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.phase(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Forger99's amplitude
model = Forger99()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(time=t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(trajectory, t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test calculation for all trajectories
amplitude = model.amplitude()
test_eq(np.all(np.isclose(amplitude, true_amplitude)), True)
# test error handling
test_fail(lambda: model.amplitude(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.amplitude(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Forger99's cbt
model = Forger99()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(), troughs)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(trajectory), troughs)), True)
# test error handling
test_fail(lambda: model.cbt(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test Forger99's dlmos
model = Forger99()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(), dlmos)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(trajectory), dlmos)), True)
# test error handling
test_fail(lambda: model.dlmos(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test that Forger99 can be entrained to a regular 24 hour light schedule for different initial conditions - Scientific test
days = 80
dt = 0.5
time = np.arange(0, 24*days, dt)
regular_schedule = LightSchedule.Regular()
light_values = regular_schedule(time)
# explore a broad space of initial conditions
amplitude_ic = np.linspace(1e-3, 1.0, 20)
phase_ic = np.linspace(-np.pi, np.pi, 20)
# convert amplitude and phase to x and xc
ic_stack = np.dstack(np.meshgrid(amplitude_ic, phase_ic)).reshape(-1, 2)
ic_stack = np.hstack((ic_stack, np.zeros((ic_stack.shape[0], 1))))
ic_x = np.sqrt(ic_stack[:, 0]) * np.cos(ic_stack[:, 1])
ic_xc = np.sqrt(ic_stack[:, 0]) * np.sin(ic_stack[:, 1])
initial_conditions = np.hstack((ic_x.reshape(-1, 1), ic_xc.reshape(-1, 1), np.zeros((ic_x.shape[0], 1))))
# transpose initial conditions
initial_conditions = initial_conditions.T
# simulate
model = Forger99()
result = model(time, initial_conditions, light_values)
# compare last ten days
t_comparison_idx = int(24 * 70 / dt)
reference_x = result.states[t_comparison_idx:, 0, 0]
reference_xc = result.states[t_comparison_idx:, 1, 0]
diff_x = result.states[t_comparison_idx:, 0, :].T - reference_x
diff_xc = result.states[t_comparison_idx:, 1, :].T - reference_xc
test_eq(np.all(np.isclose(diff_x, 0.0, atol=1e-2)), True)
test_eq(np.all(np.isclose(diff_xc, 0.0, atol=1e-2)), True)

# Hannay19

In [ ]:
# test Hannay19's constructor
model = Hannay19()
# test attributes
true_default_params = {
    'tau': 23.84, 'K': 0.06358, 'gamma': 0.024,
    'Beta1': -0.09318, 'A1': 0.3855, 'A2': 0.1977,
    'BetaL1': -0.0026, 'BetaL2': -0.957756, 'sigma': 0.0400692,
    'G': 33.75, 'alpha_0': 0.05, 'delta': 0.0075, 'p': 1.5, 'I0': 9325.0,
    'cbt_to_dlmo': 7.0}
test_eq(model._default_params, true_default_params)
true_initial_condition = np.array([0.82041911, 1.71383697, 0.52318122])
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, true_default_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 1)
for key, value in true_default_params.items():
    test_eq(getattr(model, key), value)
# test initialization with custom parameters
custom_params = {
    'tau': 1.0, 'K': 2.0, 'gamma': 3.0,
    'Beta1': 4.0, 'A1': 5.0, 'A2': 6.0,
    'BetaL1': 7.0, 'BetaL2': 8.0, 'sigma': 9.0,
    'G': 10.0, 'alpha_0': 11.0, 'delta': 12.0, 'p': 13.0, 'I0': 14.0,
    'cbt_to_dlmo': 15.0}
model = Hannay19(custom_params)
# test attributes
test_eq(model._default_params, true_default_params)
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, custom_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 1)
for key, value in custom_params.items():
    test_eq(getattr(model, key), value)

In [ ]:
# test Hannay19's integrate input handling
model = Hannay19()
time = np.array([0, 1, 2])
light_input = np.ones(len(time))
test_fail(lambda: model.integrate(time, input="1"), contains="light must be a numpy array")
test_fail(lambda: model.integrate(time, input=np.ones((1,1))), contains="light must be a 1D")
test_fail(lambda: model.integrate(time, input=np.array(["1", "2"])), contains="light must be numeric")
test_fail(lambda: model.integrate(time, input=-light_input), contains="light intensity must be nonnegative")

In [ ]:
# test Hannay19's derv
model = Hannay19()
# single batch
t = 0.0
state = np.array([0.1, 0.1, 0.1])
light = 1.0
derv_error = np.abs(np.sum(model.derv(t, state, light) - np.array([0.0007973, 0.26058529, -0.04471049])))
test_eq(derv_error < 1e-8, True)
# multiple batches
batches = 5
batch_states = np.zeros((3, batches))
for batch in range(batches):
    batch_states[:, batch] = state
batch_derv = model.derv(t, batch_states, light)
for batch in range(batches):
    derv_error = np.abs(np.sum(batch_derv[:, batch] - np.array([0.0007973, 0.26058529, -0.04471049])))
    test_eq(derv_error < 1e-8, True)

In [ ]:
# test Hannay19's phase
model = Hannay19()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(time=t)
x_vals = np.cos(model.trajectory.states[:,1])
y_vals = np.sin(model.trajectory.states[:,1])
true_phase = np.angle(x_vals + complex(0,1)*(y_vals))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(trajectory, t)
x_vals = np.cos(model.trajectory.states[:,1])
y_vals = np.sin(model.trajectory.states[:,1])
true_phase = np.angle(x_vals + complex(0,1)*(y_vals))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test calculation for all trajectories
phase = model.phase()
test_eq(np.all(np.isclose(phase, true_phase)), True)
# test error handling
test_fail(lambda: model.phase(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.phase(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Hannay19's amplitude
model = Hannay19()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(time=t)
true_amplitude = model.trajectory.states[:,0]
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(trajectory, t)
true_amplitude = model.trajectory.states[:,0]
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test calculation for all trajectories
amplitude = model.amplitude()
test_eq(np.all(np.isclose(amplitude, true_amplitude)), True)
# test error handling
test_fail(lambda: model.amplitude(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.amplitude(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Hannay19's cbt
model = Hannay19()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,1]))[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(), troughs)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,1]))[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(trajectory), troughs)), True)
# test error handling
test_fail(lambda: model.cbt(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test Hannay19's dlmos
model = Hannay19()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,1]))[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(), dlmos)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,1]))[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(trajectory), dlmos)), True)
# test error handling
test_fail(lambda: model.dlmos(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test that Hannay19 can be entrained to a regular 24 hour light schedule for different initial conditions - Scientific test
days = 30
dt = 0.1
time = np.arange(0, 24*days, dt)
regular_schedule = LightSchedule.Regular()
light_values = regular_schedule(time)
# explore a broad space of initial conditions
amplitude_ic = np.linspace(1e-2, 1.0, 20)
phase_ic = np.linspace(-np.pi, np.pi, 20)
initial_conditions = np.dstack(np.meshgrid(amplitude_ic, phase_ic)).reshape(-1, 2)
initial_conditions = np.hstack((initial_conditions, np.zeros((initial_conditions.shape[0], 1))))
# transpose initial conditions
initial_conditions = initial_conditions.T
# simulate
model = Hannay19()
result = model(time, initial_conditions, light_values)
# compare last 5 days for amplitude
t_comparison_idx = int(24 * (days - 5) / dt)
reference_R = result.states[t_comparison_idx:100:, 0, 0]
diff_R = result.states[t_comparison_idx:100:, 0, :].T - reference_R
test_eq(np.all(np.isclose(diff_R, 0.0, atol=1e-2)), True)
# compare last 5 days for phase
diff_Psi = []
for t in time[t_comparison_idx:100:]:
    phases = model.phase(t)
    reference_phase = phases[0]
    diff_Psi.append(phases - reference_phase)
test_eq(np.all(np.isclose(diff_Psi, 0.0, atol=1e-2)), True)

# Hannay19TP

In [ ]:
# test Hannay19TP's constructor
model = Hannay19TP()
# test attributes
true_default_params = {
    'tauV': 24.25, 'tauD': 24.0, 'Kvv': 0.05,
    'Kdd': 0.04, 'Kvd': 0.05, 'Kdv': 0.01,
    'gamma': 0.024, 'A1': 0.440068, 'A2': 0.159136,
    'BetaL': 0.06452, 'BetaL2': -1.38935, 'sigma': 0.0477375,
    'G': 33.75, 'alpha_0': 0.05, 'delta': 0.0075, 'p': 1.5,
    'I0': 9325.0, 'cbt_to_dlmo': 7.0}
test_eq(model._default_params, true_default_params)
true_initial_condition = np.array([0.82423745, 0.82304996, 1.75233424, 1.863457, 0.52318122])
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, true_default_params)
test_eq(model._num_states, 5)
test_eq(model._num_inputs, 1)
for key, value in true_default_params.items():
    test_eq(getattr(model, key), value)
# test initialization with custom parameters
custom_params = {
    'tauV': 1.0, 'tauD': 2.0, 'Kvv': 3.0,
    'Kdd': 4.0, 'Kvd': 5.0, 'Kdv': 6.0,
    'gamma': 7.0, 'A1': 8.0, 'A2': 9.0,
    'BetaL': 10.0, 'BetaL2': 11.0, 'sigma': 12.0,
    'G': 13.0, 'alpha_0': 14.0, 'delta': 15.0, 'p': 16.0,
    'I0': 17.0, 'cbt_to_dlmo': 18.0}
model = Hannay19TP(custom_params)
# test attributes
test_eq(model._default_params, true_default_params)
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, custom_params)
test_eq(model._num_states, 5)
test_eq(model._num_inputs, 1)
for key, value in custom_params.items():
    test_eq(getattr(model, key), value)

In [ ]:
# test Hannay19TP's integrate input handling
model = Hannay19TP()
time = np.array([0, 1, 2])
light_input = np.ones(len(time))
test_fail(lambda: model.integrate(time, input="1"), contains="light must be a numpy array")
test_fail(lambda: model.integrate(time, input=np.ones((1,1))), contains="light must be a 1D")
test_fail(lambda: model.integrate(time, input=np.array(["1", "2"])), contains="light must be numeric")
test_fail(lambda: model.integrate(time, input=-light_input), contains="light intensity must be nonnegative")

In [ ]:
# test Hannay19TP's derv
model = Hannay19TP()
# single batch
t = 0.0
state = np.array([0.1, 0.1, 0.1, 0.1, 0.1])
light = 1.0
derv_error = np.abs(np.sum(model.derv(t, state, light) - np.array([0.00063553,  0.00209955,  0.25906153,  0.26179939, -0.04471049])))
test_eq(derv_error < 1e-8, True)
# multiple batches
batches = 5
batch_states = np.zeros((5, batches))
for batch in range(batches):
    batch_states[:, batch] = state
batch_derv = model.derv(t, batch_states, light)
for batch in range(batches):
    derv_error = np.abs(np.sum(batch_derv[:, batch] - np.array([0.00063553,  0.00209955,  0.25906153,  0.26179939, -0.04471049])))
    test_eq(derv_error < 1e-8, True)

In [ ]:
# test Hannay19TP's phase
model = Hannay19TP()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(time=t)
x_vals = np.cos(model.trajectory.states[:,2])
y_vals = np.sin(model.trajectory.states[:,2])
true_phase = np.angle(x_vals + complex(0,1)*(y_vals))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(trajectory, t)
x_vals = np.cos(model.trajectory.states[:,2])
y_vals = np.sin(model.trajectory.states[:,2])
true_phase = np.angle(x_vals + complex(0,1)*(y_vals))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test calculation for all trajectories
phase = model.phase()
test_eq(np.all(np.isclose(phase, true_phase)), True)
# test error handling
test_fail(lambda: model.phase(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.phase(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Hannay19TP's amplitude
model = Hannay19TP()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(time=t)
true_amplitude = model.trajectory.states[:,0]
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(trajectory, t)
true_amplitude = model.trajectory.states[:,0]
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test calculation for all trajectories
amplitude = model.amplitude()
test_eq(np.all(np.isclose(amplitude, true_amplitude)), True)
# test error handling
test_fail(lambda: model.amplitude(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.amplitude(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Hannay19TP's cbt
model = Hannay19TP()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,2]))[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(), troughs)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,2]))[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(trajectory), troughs)), True)
# test error handling
test_fail(lambda: model.cbt(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test Hannay19TP's dlmos
model = Hannay19TP()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,2]))[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(), dlmos)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*np.cos(trajectory.states[:,2]))[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(trajectory), dlmos)), True)
# test error handling
test_fail(lambda: model.dlmos(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
#| hide
# test that Hannay19TP can be entrained to a regular 24 hour light schedule for different initial conditions - Scientific test
days = 30
dt = 0.1
time = np.arange(0, 24*days, dt)
regular_schedule = LightSchedule.Regular()
light_values = regular_schedule(time)
# explore a broad space of initial conditions
amplitude_ic = np.linspace(1e-2, 1.0, 20)
initial_conditions = np.dstack(np.meshgrid(amplitude_ic, amplitude_ic)).reshape(-1, 2)
initial_conditions = np.hstack((initial_conditions, 
                                np.zeros((initial_conditions.shape[0], 1)),
                                np.zeros((initial_conditions.shape[0], 1)),
                                np.zeros((initial_conditions.shape[0], 1))))
# transpose initial conditions
initial_conditions = initial_conditions.T
# simulate
model = Hannay19TP()
result = model(time, initial_conditions, light_values)
# compare last 5 days for amplitude
t_comparison_idx = int(24 * (days - 5) / dt)
reference_R = result.states[t_comparison_idx:100:, 0, 0]
diff_R = result.states[t_comparison_idx:100:, 0, :].T - reference_R
test_eq(np.all(np.isclose(diff_R, 0.0, atol=1e-2)), True)
# compare last 5 days for phase
diff_Psi = []
for t in time[t_comparison_idx:100:]:
    phases = model.phase(t)
    reference_phase = phases[0]
    diff_Psi.append(phases - reference_phase)
test_eq(np.all(np.isclose(diff_Psi, 0.0, atol=1e-2)), True)

# Jewett99

In [ ]:
# test Jewett99's constructor
model = Jewett99()
# test attributes
true_default_params = {
    'taux': 24.2, 'mu': 0.13, 'G': 19.875,
    'beta': 0.013, 'k': 0.55, 'q': 1.0/3,
    'I0': 9500, 'p': 0.6, 'alpha_0': 0.16,
    'phi_ref': 0.8, 'cbt_to_dlmo': 7.0}
test_eq(model._default_params, true_default_params)
true_initial_condition = np.array([-0.10097101, -1.21985662, 0.50529415])
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, true_default_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 1)
for key, value in true_default_params.items():
    test_eq(getattr(model, key), value)
# test initialization with custom parameters
custom_params = {
    'taux': 1.0, 'mu': 2.0, 'G': 3.0,
    'beta': 4.0, 'k': 5.0, 'q': 6.0,
    'I0': 7.0, 'p': 8.0, 'alpha_0': 9.0,
    'phi_ref': 10.0, 'cbt_to_dlmo': 11.0}

model = Jewett99(custom_params)
# test attributes
test_eq(model._default_params, true_default_params)
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, custom_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 1)
for key, value in custom_params.items():
    test_eq(getattr(model, key), value)

In [ ]:
# test Jewett99's integrate input handling
model = Jewett99()
time = np.array([0, 1, 2])
light_input = np.ones(len(time))
test_fail(lambda: model.integrate(time, input="1"), contains="light must be a numpy array")
test_fail(lambda: model.integrate(time, input=np.ones((1,1))), contains="light must be a 1D")
test_fail(lambda: model.integrate(time, input=np.array(["1", "2"])), contains="light must be numeric")
test_fail(lambda: model.integrate(time, input=-light_input), contains="light intensity must be nonnegative")

In [ ]:
#| hide
# test Jewett99's derv
model = Jewett99()
# single batch
t = 0.0
state = np.array([0.1, 0.1, 0.1])
light = 1.0
derv_error = np.abs(np.sum(model.derv(t, state, light) - np.array([0.03019473, -0.02595055, -0.0425285])))
test_eq(derv_error < 1e-8, True)
# multiple batches
batches = 5
batch_states = np.zeros((3, batches))
for batch in range(batches):
    batch_states[:, batch] = state
batch_derv = model.derv(t, batch_states, light)
for batch in range(batches):
    derv_error = np.abs(np.sum(batch_derv[:, batch] - np.array([0.03019473, -0.02595055, -0.0425285])))
    test_eq(derv_error < 1e-8, True)

In [ ]:
# test Jewett99's phase
model = Jewett99()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(time=t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(trajectory, t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test calculation for all trajectories
phase = model.phase()
test_eq(np.all(np.isclose(phase, true_phase)), True)
# test error handling
test_fail(lambda: model.phase(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.phase(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Jewett99's amplitude
model = Jewett99()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(time=t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(trajectory, t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test calculation for all trajectories
amplitude = model.amplitude()
test_eq(np.all(np.isclose(amplitude, true_amplitude)), True)
# test error handling
test_fail(lambda: model.amplitude(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.amplitude(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Jewett99's cbt
model = Jewett99()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs] + model.phi_ref
test_eq(np.all(np.isclose(model.cbt(), troughs)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs] + model.phi_ref
test_eq(np.all(np.isclose(model.cbt(trajectory), troughs)), True)
# test error handling
test_fail(lambda: model.cbt(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test Jewett99's dlmos
model = Jewett99()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] + model.phi_ref - 7.0
test_eq(np.all(np.isclose(model.dlmos(), dlmos)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] + model.phi_ref - 7.0
test_eq(np.all(np.isclose(model.dlmos(trajectory), dlmos)), True)
# test error handling
test_fail(lambda: model.dlmos(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test that Jewett99 can be entrained to a regular 24 hour light schedule for different initial conditions - Scientific test
days = 80
dt = 0.5
time = np.arange(0, 24*days, dt)
regular_schedule = LightSchedule.Regular()
light_values = regular_schedule(time)
# explore a broad space of initial conditions
amplitude_ic = np.linspace(1e-3, 1.0, 20)
phase_ic = np.linspace(-np.pi, np.pi, 20)
# convert amplitude and phase to x and xc
ic_stack = np.dstack(np.meshgrid(amplitude_ic, phase_ic)).reshape(-1, 2)
ic_stack = np.hstack((ic_stack, np.zeros((ic_stack.shape[0], 1))))
ic_x = np.sqrt(ic_stack[:, 0]) * np.cos(ic_stack[:, 1])
ic_xc = np.sqrt(ic_stack[:, 0]) * np.sin(ic_stack[:, 1])
initial_conditions = np.hstack((ic_x.reshape(-1, 1), ic_xc.reshape(-1, 1), np.zeros((ic_x.shape[0], 1))))
# transpose initial conditions
initial_conditions = initial_conditions.T
# simulate
model = Jewett99()
result = model(time, initial_conditions, light_values)
# compare last ten days
t_comparison_idx = int(24 * 70 / dt)
reference_x = result.states[t_comparison_idx:, 0, 0]
reference_xc = result.states[t_comparison_idx:, 1, 0]
diff_x = result.states[t_comparison_idx:, 0, :].T - reference_x
diff_xc = result.states[t_comparison_idx:, 1, :].T - reference_xc
test_eq(np.all(np.isclose(diff_x, 0.0, atol=1e-2)), True)
test_eq(np.all(np.isclose(diff_xc, 0.0, atol=1e-2)), True)

# Hilaire07

In [ ]:
# test Hilaire07's constructor
model = Hilaire07()
# test attributes
true_default_params = {
    'taux': 24.2, 'G': 37.0, 'k': 0.55, 'mu': 0.13, 'beta': 0.007, 
    'q': 1.0/3.0, 'rho': 0.032, 'I0': 9500.0, 'p': 0.5, 'a0': 0.1, 
    'phi_xcx': -2.98, 'phi_ref': 0.97, 'cbt_to_dlmo': 7.0,
    }
test_eq(model._default_params, true_default_params)
true_initial_condition = np.array([-0.0480751, -1.22504441, 0.51854818])
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, true_default_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 2)
for key, value in true_default_params.items():
    test_eq(getattr(model, key), value)
# test initialization with custom parameters
custom_params = {
    'taux': 1.0, 'mu': 2.0, 'G': 3.0,
    'beta': 4.0, 'k': 5.0, 'q': 6.0,
    'I0': 7.0, 'p': 8.0, 'alpha_0': 9.0,
    'phi_ref': 10.0, 'cbt_to_dlmo': 11.0}
model = Hilaire07(custom_params)
# test attributes
test_eq(model._default_params, true_default_params)
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, custom_params)
test_eq(model._num_states, 3)
test_eq(model._num_inputs, 2)
for key, value in custom_params.items():
    test_eq(getattr(model, key), value)

In [ ]:
# test Hilaire07's derv
model = Hilaire07()
# single batch
t = 0.0
state = np.array([0.1, 0.1, 0.1])
light = 1.0
wake = 0.0
input = np.array([light, wake])
derv_error = np.abs(np.sum(model.derv(t, state, input) - np.array([0.02610988, -0.0258909, -0.04145146])))
test_eq(derv_error < 1e-8, True)
# multiple batches
batches = 5
batch_states = np.zeros((3, batches))
for batch in range(batches):
    batch_states[:, batch] = state
batch_derv = model.derv(t, batch_states, input)
for batch in range(batches):
    derv_error = np.abs(np.sum(batch_derv[:, batch] - np.array([0.02610988, -0.0258909, -0.04145146])))
    test_eq(derv_error < 1e-8, True)

In [ ]:
# test Hilaire07's phase
model = Hilaire07()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
wake = np.zeros_like(time)
input = np.stack((light, wake), axis=1)
model(time, input=input)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(time=t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
wake = np.zeros_like(time)
wake[light > 0] = 0.0
wake[light == 0] = 1.0
input = np.stack((light, wake), axis=1)
trajectory = model(time, input=input)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(trajectory, t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test calculation for all trajectories
phase = model.phase()
test_eq(np.all(np.isclose(phase, true_phase)), True)
# test error handling
test_fail(lambda: model.phase(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.phase(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Hilaire07's amplitude
model = Hilaire07()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
wake = np.zeros_like(time)
input = np.stack((light, wake), axis=1)
model(time, input=input)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(time=t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
wake = np.zeros_like(time)
wake[light > 0] = 0.0
wake[light == 0] = 1.0
input = np.stack((light, wake), axis=1)
trajectory = model(time, input=input)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(trajectory, t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test calculation for all trajectories
amplitude = model.amplitude()
test_eq(np.all(np.isclose(amplitude, true_amplitude)), True)
# test error handling
test_fail(lambda: model.amplitude(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.amplitude(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Hilaire07's cbt
model = Hilaire07()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
wake = np.zeros_like(time)
input = np.stack((light, wake), axis=1)
trajectory = model(time, input=input)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs] + model.phi_ref
test_eq(np.all(np.isclose(model.cbt(), troughs)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
wake = np.zeros_like(time)
wake[light > 0] = 0.0
wake[light == 0] = 1.0
input = np.stack((light, wake), axis=1)
trajectory = model(time, input=input)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs] + model.phi_ref
test_eq(np.all(np.isclose(model.cbt(trajectory), troughs)), True)
# test error handling
test_fail(lambda: model.cbt(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test Hilaire07's dlmos
model = Hilaire07()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
wake = np.zeros_like(time)
input = np.stack((light, wake), axis=1)
trajectory = model(time, input=input)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] + model.phi_ref - 7.0
test_eq(np.all(np.isclose(model.dlmos(), dlmos)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
wake = np.zeros_like(time)
wake[light > 0] = 0.0
wake[light == 0] = 1.0
input = np.stack((light, wake), axis=1)
trajectory = model(time, input=input)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] + model.phi_ref - 7.0
test_eq(np.all(np.isclose(model.dlmos(trajectory), dlmos)), True)
# test error handling
test_fail(lambda: model.dlmos(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:

# test that Hilaire07 can be entrained to a regular 24 hour light schedule for different initial conditions - Scientific test
days = 80
dt = 0.5
time = np.arange(0, 24*days, dt)
regular_schedule = LightSchedule.Regular()
light_values = regular_schedule(time)
wake_values = np.zeros_like(time)
wake_values[light_values > 0] = 0.0
wake_values[light_values == 0] = 1.0
input = np.stack((light_values, wake_values), axis=1)
# explore a broad space of initial conditions
amplitude_ic = np.linspace(1e-3, 1.0, 20)
phase_ic = np.linspace(-np.pi, np.pi, 20)
# convert amplitude and phase to x and xc
ic_stack = np.dstack(np.meshgrid(amplitude_ic, phase_ic)).reshape(-1, 2)
ic_stack = np.hstack((ic_stack, np.zeros((ic_stack.shape[0], 1))))
ic_x = np.sqrt(ic_stack[:, 0]) * np.cos(ic_stack[:, 1])
ic_xc = np.sqrt(ic_stack[:, 0]) * np.sin(ic_stack[:, 1])
initial_conditions = np.hstack((ic_x.reshape(-1, 1), ic_xc.reshape(-1, 1), np.zeros((ic_x.shape[0], 1))))
# transpose initial conditions
initial_conditions = initial_conditions.T
# simulate
model = Hilaire07()
result = model(time, initial_conditions, input)
# compare last ten days
t_comparison_idx = int(24 * 70 / dt)
reference_x = result.states[t_comparison_idx:, 0, 0]
reference_xc = result.states[t_comparison_idx:, 1, 0]
diff_x = result.states[t_comparison_idx:, 0, :].T - reference_x
diff_xc = result.states[t_comparison_idx:, 1, :].T - reference_xc
test_eq(np.all(np.isclose(diff_x, 0.0, atol=1e-2)), True)
test_eq(np.all(np.isclose(diff_xc, 0.0, atol=1e-2)), True)

# Skeldon23

In [ ]:
# test Skeldon23's constructor
model = Skeldon23()
# test attributes
true_default_params = {
    'mu': 17.78, 'chi': 45.0, 'H0': 13.0, 'Delta': 1.0, 'ca': 1.72,
    'tauc': 24.2, 'f': 0.99669, 'G': 19.9, 'p': 0.6, 'k': 0.55, 'b': 0.4,
    'gamma': 0.23, 'alpha_0': 0.16, 'beta': 0.013, 'I0': 9500.0,
    'kappa': 24.0 / (2.0 * np.pi),
    'c20': 0.7896, 'alpha21': -0.3912, 'alpha22': 0.7583,
    'beta21': -0.4442, 'beta22': 0.0250, 'beta23': -0.9647, 
    'S0': 0.0,
    'cbt_to_dlmo': 7.0,
    'forced_wakeup_by_light': False, 'forced_wakeup_light_threshold': 0.0,
}
test_eq(model._default_params, true_default_params)
true_initial_condition = np.array([0.23995682, -1.1547196, 0.50529415, 12.83846474])
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, true_default_params)
test_eq(model._num_states, 4)
test_eq(model._num_inputs, 1)
test_eq(model.current_sleep_state, 0.0)
test_eq(model.sleep_state, np.array([0.0]))
for key, value in true_default_params.items():
    test_eq(getattr(model, key), value)
# test initialization with custom parameters
custom_params = {
    'mu': 1.0, 'chi': 2.0, 'H0': 3.0, 'Delta': 4.0, 'ca': 5.0,
    'tauc': 6.0, 'f': 7.0, 'G': 8.0, 'p': 9.0, 'k': 10.0, 'b': 11.0,
    'gamma': 12.0, 'alpha_0': 13.0, 'beta': 14.0, 'I0': 15.0,
    'kappa': 16.0,
    'c20': 17.0, 'alpha21': 18.0, 'alpha22': 19.0,
    'beta21': 20.0, 'beta22': 21.0, 'beta23': 22.0,
    'S0': 23.0,
    'cbt_to_dlmo': 24.0,
}
model = Skeldon23(custom_params)
# test attributes
test_eq(model._default_params, true_default_params)
test_eq(model._default_initial_condition, true_initial_condition)
test_eq(model.parameters, custom_params)
test_eq(model._num_states, 4)
test_eq(model._num_inputs, 1)
test_eq(model.current_sleep_state, 23.0)
test_eq(model.sleep_state, np.array([23.0]))
for key, value in custom_params.items():
    test_eq(getattr(model, key), value)

In [ ]:
# test Skeldon23's steprk4 new sleep state calculation
model = Skeldon23()
t = 0.0
input = np.array([1.0])
dt = 0.01 
state = np.array([0.1, 0.1, 0.1, 0.1])
ground_truth_state = np.array([0.12340915, 0.07618475, 0.04766054, 0.48855561])
new_state = model.step_rk4(t, state, dt, input)
test_eq(np.all(np.isclose(new_state, ground_truth_state)), True)
# sleep to wake transition
model = Skeldon23()
model.current_sleep_state = 1.0
state = np.array([0.1, 0.1, 0.1, 0.1])
model.step_rk4(t, state, dt, input)
new_sleep_state = model.current_sleep_state
test_eq(new_sleep_state, 0.0)
# wake to sleep transition
model = Skeldon23()
model.current_sleep_state = 0.0
state = np.array([0.1, 0.1, 0.1, 16.0])
model.step_rk4(t, state, dt, input)
new_sleep_state = model.current_sleep_state
test_eq(new_sleep_state, 1.0)
# catch error if sleep state is not 0 or 1
model = Skeldon23()
model.current_sleep_state = 0.5
state = np.array([0.1, 0.1, 0.1, 0.1])
test_fail(lambda: model.step_rk4(t, state, dt, input), contains="sleep state must be 0 or 1")

In [ ]:
# test Skeldon's integrate warnings
model = Skeldon23()
time = np.arange(0, 10, 0.1)
light = np.ones_like(time)
test_fail(lambda: model(time, np.ones((4,2)), light), contains="Skeldon23 model can't be run in batch mode")

In [ ]:
# test Skeldon23's derv
model = Skeldon23()
# single batch
t = 0.0
state = np.array([0.1, 0.1, 0.1, 0.1])
light = 1.0
derv_error = np.abs(np.sum(model.derv(t, state, light) - np.array([0.02901846, -0.02013533, -0.0425285, 0.39288889])))
test_eq(derv_error < 1e-8, True)
# multiple batches
batches = 5
batch_states = np.zeros((4, batches))
for batch in range(batches):
    batch_states[:, batch] = state
batch_derv = model.derv(t, batch_states, light)
for batch in range(batches):
    derv_error = np.abs(np.sum(batch_derv[:, batch] - np.array([0.02901846, -0.02013533, -0.0425285, 0.39288889])))
    test_eq(derv_error < 1e-8, True)

In [ ]:
# test Skeldon23's phase
model = Skeldon23()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(time=t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
phase_array = np.zeros_like(time)
for idx, t in enumerate(time):
    phase_array[idx] = model.phase(trajectory, t)
true_phase = np.angle(model.trajectory.states[:,0] + complex(0,1)*(-1.0 * model.trajectory.states[:,1]))
test_eq(np.all(np.isclose(phase_array, true_phase)), True)
# test calculation for all trajectories
phase = model.phase()
test_eq(np.all(np.isclose(phase, true_phase)), True)
# test error handling
test_fail(lambda: model.phase(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.phase(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Skeldon23's amplitude
model = Skeldon23()
time = np.linspace(0, 96, 1000)
light = np.zeros_like(time)
model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(time=t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
amplitude_array = np.zeros_like(time)
for idx, t in enumerate(time):
    amplitude_array[idx] = model.amplitude(trajectory, t)
true_amplitude = np.sqrt(model.trajectory.states[:,0] ** 2 + model.trajectory.states[:,1] ** 2)
test_eq(np.all(np.isclose(amplitude_array, true_amplitude)), True)
# test calculation for all trajectories
amplitude = model.amplitude()
test_eq(np.all(np.isclose(amplitude, true_amplitude)), True)
# test error handling
test_fail(lambda: model.amplitude(1), contains="trajectory must be a DynamicalTrajectory")
test_fail(lambda: model.amplitude(trajectory, "1"), contains="time must be a float or an int")

In [ ]:
# test Skeldon23's cbt
model = Skeldon23()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(), troughs)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
troughs = time[cbt_min_idxs]
test_eq(np.all(np.isclose(model.cbt(trajectory), troughs)), True)
# test error handling
test_fail(lambda: model.cbt(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# test Skeldon23's cbt under light blips
model = Skeldon23()
dt = 0.01 # hours
time = np.arange(0, 96, dt)
light = LightSchedule.Regular()(time)

np.random.seed(0)
blip_idxs = np.arange(0, len(time), int(2 / dt))
for blip in blip_idxs:
    if np.random.rand() > 0.85:
        for i in range(20):
            light[blip + i] = light[blip] + 750

trajectory = model(time, input=light)
cbt = model.cbt()
diff_cbt = np.diff(cbt)
test_eq(np.all(diff_cbt > 13.0), True)

In [ ]:
# test Skeldon23's dlmos
model = Skeldon23()
time = np.linspace(0, 96, 2000)
light = np.zeros_like(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(), dlmos)), True)
# test passing the trajectory
light = LightSchedule.Regular()(time)
trajectory = model(time, input=light)
cbt_min_idxs = find_peaks(-1*trajectory.states[:,0])[0]
dlmos = time[cbt_min_idxs] - 7.0
test_eq(np.all(np.isclose(model.dlmos(trajectory), dlmos)), True)
# test error handling
test_fail(lambda: model.dlmos(1), contains="trajectory must be a DynamicalTrajectory")

In [ ]:
# TODO: vectorize sleep switching to be able to handle multiple initial conditions
'''
# test that Skeldon23 can be entrained to a regular 24 hour light schedule for different initial conditions - Scientific test
days = 50
dt = 0.5
time = np.arange(0, 24*days, dt)
regular_schedule = LightSchedule.Regular()
light_values = regular_schedule(time)
# explore a broad space of initial conditions
amplitude_ic = np.linspace(1e-3, 1.0, 20)
phase_ic = np.linspace(-np.pi, np.pi, 20)
# convert amplitude and phase to x and xc
ic_stack = np.dstack(np.meshgrid(amplitude_ic, phase_ic)).reshape(-1, 2)
ic_stack = np.hstack((ic_stack, 
                      np.zeros((ic_stack.shape[0], 1)),
                      np.zeros((ic_stack.shape[0], 1))))
# transpose initial conditions
initial_conditions = ic_stack.T
# simulate
model = Skeldon23()
result = model(time, initial_conditions, light_values)
# compare last 5 days for x and xc
t_comparison_idx = int(24 * (days - 5) / dt)
reference_x = result.states[t_comparison_idx:, 0, 0]
reference_xc = result.states[t_comparison_idx:, 1, 0]
diff_x = result.states[t_comparison_idx:, 0, :].T - reference_x
diff_xc = result.states[t_comparison_idx:, 1, :].T - reference_xc
test_eq(np.all(np.isclose(diff_x, 0.0, atol=1e-2)), True)
test_eq(np.all(np.isclose(diff_xc, 0.0, atol=1e-2)), True)
'''

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()